## 导入用到的库

In [ ]:
from time import sleep
from djitellopy import Tello
from cv2 import face
from math import sqrt
import cv2
import socket
import matplotlib.pyplot as plt

In [ ]:
send_data = False

## 定义对象并连接

In [ ]:
tello = Tello()
tello.connect()
tello.streamon()

## 连接Socket客户端

In [ ]:
if send_data:
    host ='192.168.3.5'
    port = 4040
    client = socket.socket(socket.AF_INET,socket.SOCK_STREAM)
    client.settimeout(180)
    client.connect((host,port))
def SendData(data):
    if send_data:
        client.send(data.encode())
    else: print(data)
        

## 图片处理

In [ ]:
SendData(str(tello.get_battery()))
SendData(str(tello.get_temperature()))
tello.takeoff()
tello.move_up(80)

In [ ]:
TelloVideo = cv2.VideoCapture("udp://@0.0.0.0:11111")
red = (0, 0, 255)
green = (0, 255, 0)
blue = (255, 0, 0)
thickness = [1, 2, 4, 8]

while True:
    if not tello.is_flying: break
    ret, frame = TelloVideo.read()
    if ret:
        frame = cv2.resize(frame, (320,240))
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        face_cascade = cv2.CascadeClassifier('cascades/data/haarcascade_frontalface_alt2.xml')
        face_rects = face_cascade.detectMultiScale(gray, scaleFactor=1.5, minNeighbors=2)

        leftright = forwardback = updown = yawvelocity = 0
        for face_rect in face_rects:
            x, y, w, h = face_rect
            points = [(x, y), (x + w, y + h), (int(x + w/2), int(y + h/2)), (160, 120)]

            cv2.rectangle(frame, points[0], points[1], blue, thickness[1])
            cv2.circle(frame, points[2], thickness[3], blue)
            cv2.circle(frame, points[3], thickness[3], green)
            cv2.line(frame, points[2], points[3], green, thickness[1])
            plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
            distance = int(sqrt(pow(int(x + w/2) - 160, 2) + pow(int(y + h/2) - 120, 2)))
            size = int(w * h)

            if distance > 50:
                med_face, med_pic = points[2], points[3]
                output = ''
                if med_face[0] - med_pic[0] > 0:
                    v = med_pic[0] - med_face[0]
                    leftright = v
                    yawvelocity = int(-v / 2)
                    output += '右'
                else:
                    v = med_face[0] - med_pic[0]
                    leftright = v
                    yawvelocity = int(-v / 2)
                    output += '左'
                if med_face[1] - med_pic[1] > 0:
                    v = med_pic[1] - med_face[1]
                    updown = int(v / 2)
                    output += '下'
                else:
                    v = med_face[1] - med_pic[1]
                    updown = int(v / 2)
                    output += '上'
                SendData(f'头像在画面的{output}方，{distance}')
            else:
                SendData(str(distance))
                leftright = updown = yawvelocity = 0
            if size > 8000:
                forwardback = int(size / 2000)
                SendData(f'头像偏近：{size}')
            elif size < 4000:
                forwardback = int(-size / 2000)
                SendData(f'头像偏远：{size}')
            else:
                SendData(str(size))
                forwardback = 0
        tello.send_rc_control(leftright, forwardback, updown, yawvelocity)
    sleep(1/2)